In [2]:
#%pip install wget
#%pip install torch
import data_rnn
import numpy as np
import torch

In [3]:
(x_train, y_train), (x_val, y_val), (i2w, w2i), numcls = data_rnn.load_imdb(final=False)

Inspecting the data

In [4]:
x_train[0], y_train[0] # 0 is positive, 1 is negative

([14, 19, 9, 379, 22, 11, 50, 52, 53, 290], 1)

In [5]:
[i2w[word] for word in x_train[0]]
len(i2w)

99430

In [6]:
lengths = [len(l) for l in x_train]
print(np.min(lengths), np.max(lengths), np.average(lengths))

10 2514 240.6318


In [7]:
w2i['.pad'], w2i['.start'], w2i['.end'], w2i['.unk']

(0, 1, 2, 3)

In [8]:
def pad(seq, pad_length):
    padded = np.zeros(pad_length) # 0 is for padding
    padded[0:len(seq)] = seq
    return torch.tensor(padded, dtype=torch.long)
    

In [9]:
padded = pad(x_train[0], 12).reshape(1,-1)
padded.shape

torch.Size([1, 12])

In [22]:
# Take a tensor x, and return max across time dimension
def MaxPoolTime(x):
    return torch.amax(x, dim=1)

class q2_baseline(torch.nn.Module):
    def __init__(self, batch_size=1):
        super().__init__()
        timestep = 12
        numcls = 2
        hidden = 300
        embedding_size = 300
        n_embeddings = len(i2w)
        self.emb = torch.nn.Embedding(n_embeddings, embedding_size, padding_idx=0)
        self.fc1 = torch.nn.Linear(embedding_size, hidden)
        self.fc2 = torch.nn.Linear(hidden, numcls)
    
    def forward(self, x):
        x = self.emb(x)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = MaxPoolTime(x)
        x = self.fc2(x)

        return x

In [11]:
baseline_model = q2_baseline()
y = baseline_model.forward(padded)
torch.nn.functional.softmax(y)

x before:torch.Size([1, 12, 300])
x after:torch.Size([1, 300])


C:\Users\Dany\AppData\Local\Temp\ipykernel_23664\2300644675.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  torch.nn.functional.softmax(y)


tensor([[0.7417, 0.2583]], grad_fn=<SoftmaxBackward0>)

In [12]:
# pad all
padding_size = np.max(lengths)
padded_train = []
padded_val = []
for x in x_train:
    padded_train.append(pad(x, padding_size))
for x in x_val:
    padded_val.append(pad(x, padding_size))


In [13]:
batch_size = 1

trainloader = torch.utils.data.DataLoader(padded_train, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(padded_val, batch_size=batch_size, shuffle=False)


In [18]:
import torch.optim as optim
# setup training loop

optimizer = optim.Adam(baseline_model.parameters(), lr=0.001)

In [23]:
baseline_model = q2_baseline(1)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, batch in enumerate(trainloader, 0):

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = baseline_model(batch)
        loss = torch.nn.functional.cross_entropy(outputs, torch.tensor([y_train[i]]))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 0.717
[1,  4000] loss: 0.704
[1,  6000] loss: 0.699
[1,  8000] loss: 0.700


KeyboardInterrupt: 